<a href="https://colab.research.google.com/github/bhaveshgupta01/BCancerGogo/blob/main/GOGOvgg19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.applications.vgg16 import preprocess_input


# Image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 16

def label_images(directory, target_size=(img_width, img_height)):
    images = []
    labels = []
    class_labels = {'malignant': 0, 'benign': 1, 'normal': 2}

    for class_label, class_index in class_labels.items():
        class_path = os.path.join(directory, class_label)
        for filename in os.listdir(class_path):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                file_path = os.path.join(class_path, filename)
                image = cv2.imread(file_path)  # You can use PIL.Image.open() as well
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB format
                image = cv2.resize(image, target_size)  # Resize the image to target size
                image = preprocess_input(image)  # Preprocess the image for VGG16
                images.append(image)
                labels.append(class_index)

    return np.array(images), np.array(labels)


# Example usage:
directory_path = '/content/drive/MyDrive/MIASdata'
X, y = label_images(directory_path, target_size=(img_width, img_height))

# X contains the resized images, and y contains the corresponding labels
print(f"Total Images: {len(X)}")
print(f"Shape of an Image: {X[0].shape}")
print(f"Labels: {y}")


Total Images: 321
Shape of an Image: (224, 224, 3)
Labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [ ]:
from sklearn.model_selection import train_test_split

# Assuming X and y are the images and labels obtained from the previous code
# X, y = label_images(directory_path)

# Split the data into 90% training and 10% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Print the sizes of the training and testing sets
print(f"Training Set: {len(X_train)} samples")
print(f"Testing Set: {len(X_test)} samples")


Training Set: 288 samples
Testing Set: 33 samples


In [ ]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import models, layers
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold
import numpy as np

# Assuming you have three classes (malignant, benign, normal)
num_classes = 3
img_width, img_height = 224, 224  # Adjust these dimensions based on your data

def build_vgg19_model(learn_rate=1e-4, momentum=0.9):
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

    # Freeze all layers except the last three
    for layer in base_model.layers[:-3]:
        layer.trainable = False

    model = models.Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(num_classes, activation='softmax'))

    optimizer = SGD(learning_rate=learn_rate, momentum=momentum)

    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Build a new VGG19 model
vgg19_model = build_vgg19_model()

# Number of folds for k-fold cross-validation
num_folds = 5
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

for fold, (train_index, val_index) in enumerate(kfold.split(X, y), 1):
    X_train_fold, X_val_fold = X[train_index], X[val_index]
    y_train_fold, y_val_fold = y[train_index], y[val_index]

    # Convert labels to one-hot encoding
    y_train_fold_one_hot = to_categorical(y_train_fold, num_classes=num_classes)
    y_val_fold_one_hot = to_categorical(y_val_fold, num_classes=num_classes)

    # Train the VGG19 model
    class_weights = {0: 4.0, 1: 4.0, 2: 1.0}  # Adjust the weights based on class imbalance

    history = vgg19_model.fit(
        X_train_fold,
        y_train_fold_one_hot,
        epochs=10,
        validation_data=(X_val_fold, y_val_fold_one_hot),
        batch_size=32,
        verbose=1,
        class_weight=class_weights
    )

    # Evaluate the model on the validation set
    val_loss, val_acc = vgg19_model.evaluate(X_val_fold, y_val_fold_one_hot, verbose=0)
    print(f"Validation Accuracy for Fold {fold}: {val_acc * 100:.2f}%")


80134624/80134624 [==============================] - 1s 0us/step
Epoch 1/10
8/8 [==============================] - 313s 40s/step - loss: 3.1583 - accuracy: 0.3242 - val_loss: 3.5590 - val_accuracy: 0.2769
Epoch 2/10
8/8 [==============================] - 282s 37s/step - loss: 1.1666 - accuracy: 0.6172 - val_loss: 2.4153 - val_accuracy: 0.4000
Epoch 3/10
8/8 [==============================] - 306s 40s/step - loss: 0.6107 - accuracy: 0.7969 - val_loss: 2.5222 - val_accuracy: 0.4308
Epoch 4/10
8/8 [==============================] - 315s 41s/step - loss: 0.4389 - accuracy: 0.8750 - val_loss: 2.0625 - val_accuracy: 0.4308
Epoch 5/10
8/8 [==============================] - 306s 39s/step - loss: 0.3174 - accuracy: 0.9414 - val_loss: 1.7532 - val_accuracy: 0.4462
Epoch 6/10
8/8 [==============================] - 307s 40s/step - loss: 0.2297 - accuracy: 0.9805 - val_loss: 1.5886 - val_accuracy: 0.4615
Epoch 7/10
8/8 [==============================] - 306s 40s/step - loss: 0.2108 - accuracy: 0.98

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_test_one_hot=to_categorical(y_test, num_classes=num_classes)

# Function to calculate and print evaluation metrics
def evaluate_model_multi_class(model, X, y_true):
    # Predictions
    y_pred = model.predict(X)

    # Convert one-hot encoding to class labels
    y_true_labels = np.argmax(y_true, axis=1)
    y_pred_labels = np.argmax(y_pred, axis=1)

    # Calculate accuracy
    accuracy = accuracy_score(y_true_labels, y_pred_labels)
    print(f"Accuracy: {accuracy * 100:.2f}%")

    # Confusion Matrix
    cm = confusion_matrix(y_true_labels, y_pred_labels)
    print("Confusion Matrix:")
    print(cm)

    # Classification Report
    print("Classification Report:")
    print(classification_report(y_true_labels, y_pred_labels))

# Assuming you have trained the model 'inception_model' and loaded the test set 'X_val_fold', 'y_val_fold_one_hot'
evaluate_model_multi_class(vgg19_model, X_test, y_test_one_hot)


2/2 [==============================] - 27s 741ms/step
Accuracy: 100.00%
Confusion Matrix:
[[ 4  0  0]
 [ 0  9  0]
 [ 0  0 20]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         4
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        20

    accuracy                           1.00        33
   macro avg       1.00      1.00      1.00        33
weighted avg       1.00      1.00      1.00        33

